Importación y Carga de Datos

In [10]:
# Importaciones
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC  # Importa el clasificador SVM en lugar de SVR, ya que estás haciendo clasificación
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

df = pd.read_csv('telecom_customer_churn.csv')

Preprocesamiento

In [11]:
# 1. Preprocesamiento
# a. Eliminar entradas con el estado "Joined"
df = df[df['Customer Status'] != 'Joined']

# b. Crear la columna 'Churn' y asignar 1 si 'Customer Status' es 'Churned', de lo contrario 0
df['Churn'] = df['Customer Status'].apply(lambda x: 1 if x == 'Churned' else 0)

# c. Eliminar columnas con más del 50% de datos faltantes
threshold = int(0.5 * len(df))
df = df.dropna(thresh=threshold, axis=1)

# d. Reemplazar valores atípicos por la media
for col in df.select_dtypes(include=['float64', 'int64']):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    df[col] = df[col].apply(lambda x: df[col].mean() if (x < (Q1 - 1.5 * IQR)) or (x > (Q3 + 1.5 * IQR)) else x)

# e. Convertir variables categóricas a numéricas y llenar valores faltantes
for column in df.columns:
    if df[column].dtype == 'object' and column != 'Customer Status':
        # Llenar valores faltantes con la moda para columnas categóricas
        df[column].fillna(df[column].mode()[0], inplace=True)
        # Convertir variables categóricas a numéricas
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])
    elif df[column].dtype in ['int64', 'float64']:
        # Llenar valores faltantes con la mediana para columnas numéricas
        df[column].fillna(df[column].median(), inplace=True)

# Eliminar la columna 'Customer Status' antes de normalizar
df = df.drop(columns=['Customer Status'])

# f. Normalización
# Seleccionar solo las columnas que no son 'Churn' para la normalización
cols_to_scale = df.columns.difference(['Churn'])
scaler = StandardScaler()
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

Selección de características

In [12]:
# 2. Selección de características
xgb_for_feature_selection = XGBClassifier(
    objective='binary:logistic', 
    random_state=42, 
    use_label_encoder=False, 
    eval_metric='logloss'
)
xgb_for_feature_selection.fit(df.drop('Churn', axis=1), df['Churn'])
threshold = 0.01  
selected_features = df.drop('Churn', axis=1).columns[(xgb_for_feature_selection.feature_importances_ > threshold)].tolist()


Balanceo con SMOTE

In [13]:
# 3. Balanceo con SMOTE
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(df[selected_features], df['Churn'])

Balanceo

In [14]:
# 4. División de Conjunto
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

Entrenamiento de SVR con búsqueda de hiperparámetros

In [15]:
# 5. Entrenamiento de SVR con búsqueda de hiperparámetros usando GridSearchCV
param_grid = {
    'C': [0.1, 1, 10, 100],  # Parámetros de regularización
    'gamma': [1, 0.1, 0.01, 0.001],  # Parámetro de kernel para 'rbf', 'poly' y 'sigmoid'
    'kernel': ['rbf', 'linear']  # Tipo de kernel
}

grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=3)
grid.fit(X_train, y_train)

# Imprimir los mejores parámetros encontrados
print(grid.best_params_)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.570 total time=   1.2s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.565 total time=   1.2s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.581 total time=   1.1s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.602 total time=   1.3s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.591 total time=   1.2s
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.823 total time=   0.4s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.823 total time=   0.4s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.820 total time=   0.4s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.814 total time=   0.4s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.830 total time=   0.4s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.832 total time=   0.8s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf

Evaluación del modelo

In [16]:
# 6. Evaluación del modelo con los mejores hiperparámetros
best_svr_model = grid.best_estimator_
y_pred = best_svr_model.predict(X_test)

# Calcular la matriz de confusión, y generar el reporte de clasificación
conf_matrix = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Imprimir las métricas
print(conf_matrix)
print("Reporte de clasificación:\n", report)

[[825 129]
 [ 54 880]]
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.94      0.86      0.90       954
           1       0.87      0.94      0.91       934

    accuracy                           0.90      1888
   macro avg       0.91      0.90      0.90      1888
weighted avg       0.91      0.90      0.90      1888



Validación cruzada

In [17]:
# 7. Validación Cruzada
scores = cross_val_score(best_svr_model, X_train, y_train, cv=5, scoring='accuracy')
# Imprimir el promedio de los scores de la validación cruzada
print("Accuracy promedio con Validación Cruzada:", scores.mean())

Accuracy promedio con Validación Cruzada: 0.8899637536651751


In [18]:
print(grid.best_estimator_)

SVC(C=10, gamma=0.1)
